Определим кол-во активных пользователей приложения в различные временные интервалы и получим актуальные аудиторные данные.

In [ ]:
# DAU

SELECT toStartOfDay(toDateTime(time)) AS __timestamp,
       count(DISTINCT user_id) AS "COUNT_DISTINCT(user_id)"
FROM simulator_20221220.feed_actions
WHERE time >= toDateTime('2022-12-01 00:00:00')
  AND time < toDateTime('2023-01-01 00:00:00')
GROUP BY toStartOfDay(toDateTime(time))
ORDER BY "COUNT_DISTINCT(user_id)" DESC
LIMIT 50000;

In [ ]:
# WAU
    
SELECT toMonday(toDateTime(time)) AS __timestamp,
       count(DISTINCT user_id) AS "COUNT_DISTINCT(user_id)"
FROM simulator_20221220.feed_actions
WHERE time >= toDateTime('2022-12-01 00:00:00')
  AND time < toDateTime('2023-01-01 00:00:00')
GROUP BY toMonday(toDateTime(time))
ORDER BY "COUNT_DISTINCT(user_id)" DESC
LIMIT 50000;

In [ ]:
# MAU :
    
SELECT toStartOfMonth(toDateTime(time)) AS __timestamp,
       COUNT(DISTINCT user_id) AS "Уникальные пользователи"
FROM simulator_20221220.feed_actions
GROUP BY toStartOfMonth(toDateTime(time))
ORDER BY "Уникальные пользователи" DESC
LIMIT 50000;

Далее строим график по эволюции основных событий (в нашем случае - лайков и просмотров)

In [ ]:
SELECT toStartOfDay(toDateTime(time)) AS __timestamp,
       action AS action,
       count(user_id) AS "COUNT(user_id)"
FROM simulator_20221220.feed_actions
WHERE time >= toDateTime('2022-12-01 00:00:00')
  AND time < toDateTime('2023-01-01 00:00:00')
GROUP BY action,
         toStartOfDay(toDateTime(time))
ORDER BY "COUNT(user_id)" DESC
LIMIT 50000;

Строим график CTR

In [ ]:
SELECT toStartOfDay(toDateTime(time)) AS __timestamp,
       countIf(user_id, action = 'like') / countIf(user_id, action = 'view') AS "CTR"
FROM simulator_20221220.feed_actions
WHERE time >= toDateTime('2022-12-01 00:00:00')
  AND time < toDateTime('2023-01-01 00:00:00')
GROUP BY toStartOfDay(toDateTime(time))
ORDER BY "CTR" DESC
LIMIT 50000;

Определим ТОП - 50 постов по количеству просмотров. 
Для каждого поста из ТОПа отразим дополнительно кол-во лайков, CTR и охват аудитории.

In [ ]:
SELECT post_id AS post_id,
       countIf(action = 'view') AS "Просмотры",
       countIf(action = 'like') AS "Лайки",
       countIf(action = 'like') / countIf(action = 'view') AS "CTR",
       count(DISTINCT user_id) AS "Охват"
FROM simulator_20221220.feed_actions
GROUP BY post_id
ORDER BY "Просмотры" DESC
LIMIT 50;

Так же на наш основной дашборд поместим фильтр по полу и операционной системе с возможностью выбора интервала времени.

Дополнительно создадим еще один дашборд, на который разместим вспомогательную справочную информацию:

Узнаем какое среднее число событий приходится на каждого уникального юзера нашего приложения за все время:
    

In [ ]:
SELECT min(avg_actions) AS "MIN(avg_actions)"
FROM
  (select round(avg(actions_count), 2) as avg_actions
   from
     (SELECT count(action) as actions_count,
             user_id
      FROM simulator.feed_actions
      group by user_id,
               toDate(time)) t) AS virtual_table
LIMIT 50000;

![](pics\simulator\среднее-кол-во-действий-пользователя-2023-01-06T10-46-44.612Z.jpg)


Определим локации самых активных юзеров:

In [ ]:
select
  distinct t1.user_id,
  actions_count,
  country,
  city
from
  (
    SELECT
      count(action) as actions_count,
      user_id
    FROM
      simulator.feed_actions
    group by
      user_id
    order by
      actions_count desc
    limit
      50
  ) t1
  join (
    select
      user_id,
      country,
      city
    from
      feed_actions
  ) t2 using(user_id)

![](pics\simulator\топ-50-активных-юзеров-2023-01-06T11-27-37.562Z.jpg)

А так же составим список Топ-50 городов по числу событий:

In [ ]:
SELECT city AS city,
       COUNT(action) AS "События",
       COUNT(action) AS "Доля %"
FROM simulator_20221220.feed_actions
GROUP BY city
ORDER BY "Доля %" DESC
LIMIT 10000;

![](pics\simulator\топ-городов-по-активности-2023-01-06T12-04-48.919Z.jpg)

Определим, как меняется активность наших пользователей в течение суток за последнюю неделю:

In [ ]:
SELECT toStartOfHour(toDateTime(time)) AS __timestamp,
       action AS action,
       count(action) AS "COUNT(action)"
FROM simulator_20221220.feed_actions
WHERE time >= toDateTime('2022-12-30 00:00:00')
  AND time < toDateTime('2023-01-06 00:00:00')
GROUP BY action,
         toStartOfHour(toDateTime(time))
ORDER BY "COUNT(action)" DESC
LIMIT 50000;

![](pics\simulator\дневная-активность-2023-01-06T12-19-46.140Z.jpg)

Из графика мы видим, что спад активности наблюдается в период с 3 до 4 часов ночи,
а пики активности приходятся на вечернее время с 19 до 21 часов.

Определим распределение наших пользователей по странам и выясним, в киких странах проживает наибольшее число юзеров:
    

In [ ]:
SELECT country AS country,
       count(DISTINCT user_id) AS "COUNT_DISTINCT(user_id)"
FROM simulator_20221220.feed_actions
GROUP BY country
ORDER BY "COUNT_DISTINCT(user_id)" DESC
LIMIT 50;

![](pics\simulator\география-юзеров-2023-01-06T14-05-28.894Z.jpg)

В конечном итоге собираем наши вспомогательные метрики воедино и получаем вот такой дашборд с дополнительными показателямиЖ

![](pics\simulator\Лента_доп.png)

 Создадим еще один дошборд с опреативными данными, чтобы иметь возможность наблюдать и сравнивать  
 текущие данные с какими-то периодами в прошлом, тем самым выявляя тенденции или аномалии.



Отобразим кол-во событий сегодня, вчера и неделю назад. В каестве временной шкалы зададим период в 15 минут:

In [ ]:
SELECT toDateTime(intDiv(toUInt32(toDateTime(time)), 900)*900) AS __timestamp,
       COUNT(user_id) AS "События"
FROM simulator_20221220.feed_actions
WHERE time >= toDateTime('2023-01-06 00:00:00')
  AND time < toDateTime('2023-01-07 00:00:00')
GROUP BY toDateTime(intDiv(toUInt32(toDateTime(time)), 900)*900)
ORDER BY "События" DESC
LIMIT 50000;

![](pics\simulator\все-события-2023-01-06T14-31-38.908Z.jpg)

Свал графика с текущими показателями обусловлен тем, что за настоящий период еще не все данные доступны для обработки.
На графике видно, что кол-во событий растет, что логично, так как график DAU из основного 
дашборда ленты новостей  демонстрировал нам рост кол-ва активных пользователей нашего приложения.

 На дополнительный дашборд разместим некоторые продуктовые метрики по аудитории для углубленного анализа

Посмотрим какое количество пользователей из числа тех, кто начал пользоваться нашим продуктом в последние 20 дней, 
продолжаеют им пользоваться, иными словами, посмотрим как новые пользователи удерживаются нашим приложением, 
для этого визуализируем график Retention.

In [ ]:
SELECT date AS date,
       start_date AS start_date,
       max(active_users) AS "MAX(active_users)"
FROM
  (select toString(date) as date,
          toString(start_date) as start_date,
          count(user_id) as active_users
   from
     (select user_id,
             min(toDate(time)) as start_date
      from simulator_20221220.feed_actions
      group by user_id
      having start_date >= today() - 20) t1
   join
     (select distinct user_id,
                      toDate(time) as date
      from simulator_20221220.feed_actions) t2 using(user_id)
   group by date, start_date) AS virtual_table
GROUP BY date, start_date
LIMIT 1000;

Все графики и скриншоты дашбордов для вышенаписанных запросов находятся в папке visalizations